### Auto

In [45]:
import numpy as np

import pandas as pd

In [46]:
auto = pd.read_excel("/mnt/e/Kerjaan/Kuliah/S2/thesis_3/data/auto/raw/auto.xls")
auto_numpy = auto.to_numpy()[:, 1:].astype(float)
np.save("/mnt/e/Kerjaan/Kuliah/S2/thesis_3/data/auto/preprocessed/ts", auto_numpy)

### Car Parts

In [13]:
import numpy as np
import pandas as pd

In [15]:
carparts_df = pd.read_csv("/mnt/e/Kerjaan/Kuliah/S2/thesis_3/data/carparts/raw/carparts.csv").dropna()
carparts_numpy = carparts_df.to_numpy()[:, 1:]
np.save("/mnt/e/Kerjaan/Kuliah/S2/thesis_3/data/carparts/preprocessed/ts", carparts_numpy)

### RAF

In [1]:
import json

import numpy as np

In [21]:
with open("../data/raf/raw/train/data.json") as raf_train:
    raf_train = json.load(raf_train)

with open("../data/raf/raw/test/data.json") as raf_test:
    raf_test = json.load(raf_test)

In [50]:
targets = []
for train, test in zip(raf_train, raf_test):
    test_target = test["target"]
    targets.append(test_target)

raf_numpy = np.array(targets)
np.save("../data/raf/preprocessed/ts", raf_numpy)

## Exploration

In [1]:
import numpy as np

In [61]:
auto = np.load("../data/auto/preprocessed/ts.npy")
raf = np.load("../data/raf/preprocessed/ts.npy")
carparts = np.load("../data/carparts/preprocessed/ts.npy")

In [70]:
def create_summaries(numpy_data):
    ps, covs = [], []
    dtypes = {
        "smooth": 0,
        "intermittent": 0,
        "lumpy": 0,
        "erratic": 0,
    }
    for i in range(numpy_data.shape[0]):
        single_numpy_data = numpy_data[i, :]
        p = len(single_numpy_data)/len(single_numpy_data[single_numpy_data!=0])
        cov = (np.std(single_numpy_data[single_numpy_data!=0])/np.mean(single_numpy_data[single_numpy_data!=0]))**2

        ps.append(p)
        covs.append(cov)

        if p < 1.32 and cov < 0.49:
            dtypes["smooth"] += 1
        elif p < 1.32 and cov >= 0.49:
            dtypes["erratic"] += 1
        elif p >= 1.32 and cov < 0.49:
            dtypes["intermittent"] += 1
        else:
            dtypes["lumpy"] += 1

    return_str = """mean_p: {} \nmean_covs: {} \nts_classification: \n{}
    """.format(
        np.mean(ps),
        np.mean(covs),
        "\n".join([f"{key}: {round(val/sum(dtypes.values()) * 100, 2)}" for key, val in dtypes.items()])
    )
    return return_str

In [71]:
print("Auto")
print(create_summaries(auto))
print("RAF")
print(create_summaries(raf))
print("M5")
print(create_summaries(m5))
print("Car Parts")
print(create_summaries(carparts))

Auto
mean_p: 1.3200429161481793 
mean_covs: 0.41191122805426433 
ts_classification: 
smooth: 41.37
intermittent: 35.8
lumpy: 10.23
erratic: 12.6
    
RAF
mean_p: 11.135707019708878 
mean_covs: 0.6306787975894361 
ts_classification: 
smooth: 0.0
intermittent: 58.1
lumpy: 41.9
erratic: 0.0
    
M5
mean_p: 4.873108028920366 
mean_covs: 0.39587359693586416 
ts_classification: 
smooth: 4.39
intermittent: 75.27
lumpy: 18.96
erratic: 1.38
    
Car Parts
mean_p: 7.49067238064312 
mean_covs: 0.27348444102431807 
ts_classification: 
smooth: 0.0
intermittent: 86.57
lumpy: 13.43
erratic: 0.0
    


Auto
mean_p: 1.3200429161481793 
mean_covs: 0.8615525556741409 
ts_classification: 
smooth: 656
intermittent: 4
lumpy: 1377
erratic: 963
    


RAF
mean_p: 11.135707019708878 
mean_covs: 16.61155449154367 
ts_classification: 
smooth: 0
intermittent: 0
lumpy: 5000
erratic: 0
    
